In [75]:
import os
import json
import pandas as pd



In [ ]:
tour_name = "girodabruzzo"
stage_number = "2"

In [81]:
def flatten_group(gruppo, group_index, timestamp, average_speed):
    return {
        "timestamp": timestamp,
        "group_index": group_index,
        "average_speed": average_speed,
        "group_type": gruppo.get("tipo"),
        "km": gruppo.get("km"),
        "time_gap": gruppo.get("distacco"),
        "num_riders": len(gruppo.get("corridori", [])),
        
        "riders": [
            f"{c['nome']} {c['cognome']} ({c['team'].strip()})"
            for c in gruppo.get("corridori", [])
        ]
    }

def combine_to_timeline(directory):
    timeline = []

    for file in sorted(os.listdir(directory)):
        if file.endswith("Z.json"):
            with open(os.path.join(directory, file), "r") as f:
                data = json.load(f)
                timestamp = data.get("timestamp")
                average_speed = data.get("velocita_media")
                linea = data.get("linea")
                gruppi = linea.get("gruppi", [])
                print(f"Processing file: {file} with timestamp: {timestamp}")
                print(gruppi)
                

                for i, gruppo in enumerate(gruppi):
                    timeline.append(flatten_group(gruppo, i, timestamp, average_speed))
                    print(f"Processed group {i} from file {file}")

    df = pd.DataFrame(timeline)
    return df



In [150]:
# Example:
timeline_df = combine_to_timeline(f"data/live_data/raw/{tour_name}/stage-{stage_number}")

Processing file: 20250415T112244Z.json with timestamp: 20250415T112244Z
[{'tipo': 'GRUPPO_TESTA', 'km': '60', 'distacco': False, 'corridori': [{'id': 'ATL_0027', 'nome': 'Cristian', 'cognome': 'REMELLI', 'team': 'GEF '}, {'id': 'ATL_0062', 'nome': 'Diego', 'cognome': 'BRACALENTE', 'team': 'MBH'}, {'id': 'ATL_0117', 'nome': 'Matteo', 'cognome': 'ZURLO', 'team': 'PAD'}]}, {'tipo': 'GRUPPO_INSEGUITORI', 'km': '60', 'distacco': "+ 03' 00''", 'corridori': [{'id': 'ATL_0083', 'nome': 'Sampo', 'cognome': 'MALINEN', 'team': 'MIG'}]}, {'tipo': 'GRUPPO_INSEGUITORI', 'km': '60', 'distacco': "+ 04' 20''", 'corridori': [{'id': 'GRUPPO_60_2', 'nome': 'Group', 'cognome': '', 'team': ''}]}]
Processed group 0 from file 20250415T112244Z.json
Processed group 1 from file 20250415T112244Z.json
Processed group 2 from file 20250415T112244Z.json
Processing file: 20250415T112744Z.json with timestamp: 20250415T112744Z
[{'tipo': 'GRUPPO_TESTA', 'km': '62', 'distacco': False, 'corridori': [{'id': 'ATL_0027', 'nom

In [152]:
timeline_df

,timestamp,group_index,average_speed,group_type,km,time_gap,num_riders,riders
0,20250415T112244Z,0,"43,4 Km/h",GRUPPO_TESTA,60,False,3,"Cristian REMELLI (GEF), Diego BRACALENTE (MBH)..."
1,20250415T112244Z,1,"43,4 Km/h",GRUPPO_INSEGUITORI,60,+ 03' 00'',1,Sampo MALINEN (MIG)
2,20250415T112244Z,2,"43,4 Km/h",GRUPPO_INSEGUITORI,60,+ 04' 20'',1,Large group
3,20250415T112744Z,0,"42,3 Km/h",GRUPPO_TESTA,62,False,3,"Cristian REMELLI (GEF), Diego BRACALENTE (MBH)..."
4,20250415T112744Z,1,"42,3 Km/h",GRUPPO_INSEGUITORI,62,+ 03' 12'',1,Sampo MALINEN (MIG)
...,...,...,...,...,...,...,...,...
62,20250415T132352Z,1,"42,4 Km/h",GRUPPO_INSEGUITORI,137,+ 30'',1,Large group
63,20250415T132553Z,0,"42,4 Km/h",GRUPPO_TESTA,147,False,1,Large group
64,20250415T132753Z,0,"42,4 Km/h",GRUPPO_TESTA,147,False,1,Large group
65,20250415T132953Z,0,"42,7 Km/h",GRUPPO_TESTA,150,False,1,Large group


In [154]:
timeline_df["average_speed"] = (
    timeline_df["average_speed"]
    .str.replace(" Km/h", "", regex=False)  # remove unit
    .str.replace(",", ".", regex=False)     # convert comma to dot
    .astype(float)                          # convert to float
)


In [155]:

# Update group_type values based on group_index
timeline_df["group_type"] = "Group " + (timeline_df["group_index"] + 1).astype(str)

In [151]:
# Update riders column
timeline_df["riders"] = timeline_df["riders"].apply(
    lambda x: "Large group" if x == ["Group  ()"] else ", ".join(x)
)

In [156]:
import re

def parse_time_gap(time_str):
    """Convert a '+ 03' 20''' string to minutes as float."""
    if not isinstance(time_str, str):
        return 0.0
    match = re.match(r"\+?\s*(\d+)'(?:\s*(\d+)'')?", time_str)
    if not match:
        return 0.0
    minutes = int(match.group(1))
    seconds = int(match.group(2)) if match.group(2) else 0
    return minutes + seconds / 60

# Merge or look up average speed per timestamp (you probably already have this)
# Assume timeline_df has "velocita_media" in km/h
timeline_df["time_gap_min"] = timeline_df["time_gap"].apply(parse_time_gap)

# Convert km to float (may currently be string)
timeline_df["km"] = timeline_df["km"].astype(float)

# Create adjusted km column
timeline_df["km_adjusted"] = timeline_df["km"]  # default: no change
mask = timeline_df["group_index"] > 0

# Estimate distance behind, subtract from leader's km
timeline_df.loc[mask, "km_adjusted"] = (
    timeline_df.loc[mask, "km"] -
    (timeline_df.loc[mask, "time_gap_min"] / 60) * timeline_df.loc[mask, "average_speed"]
)

In [166]:
timeline_df

,timestamp,group_index,average_speed,group_type,km,time_gap,num_riders,riders,time_gap_min,km_adjusted
2,20250415T112244Z,2,43.4,Group 3,60.0,+ 04' 20'',1,Large group,4.333333,56.865556
1,20250415T112244Z,1,43.4,Group 2,60.0,+ 03' 00'',1,Sampo MALINEN (MIG),3.000000,57.830000
5,20250415T112744Z,2,42.3,Group 3,62.0,+ 03' 53'',1,Large group,3.883333,59.262250
8,20250415T113016Z,2,42.3,Group 3,62.0,+ 03' 53'',1,Large group,3.883333,59.262250
7,20250415T113016Z,1,42.3,Group 2,62.0,+ 03' 12'',1,Sampo MALINEN (MIG),3.200000,59.744000
...,...,...,...,...,...,...,...,...,...,...
61,20250415T132352Z,0,42.4,Group 1,137.0,False,8,"Louis Du Bouisson MEINTJES (IWA), Luca CRETTI ...",0.000000,137.000000
63,20250415T132553Z,0,42.4,Group 1,147.0,False,1,Large group,0.000000,147.000000
64,20250415T132753Z,0,42.4,Group 1,147.0,False,1,Large group,0.000000,147.000000
65,20250415T132953Z,0,42.7,Group 1,150.0,False,1,Large group,0.000000,150.000000


In [158]:
timeline_df.to_csv(f"data/live_data/{tour_name}/stage_{stage_number}_basic_timeline.csv", index=False)

In [159]:
profile_df = pd.read_csv(f"data/gpx_parsed/giro-abruzzo-2025-stage-{stage_number}.csv")
profile_df

,latitude,longitude,elevation,time,name,link,distance,elevation_diff,cum_elevation,cum_distance
0,42.09387,14.63236,82.0,2025-04-15 14:36:39+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,0.000000,0.0,0.0,0.000000
1,42.09623,14.63277,83.0,2025-04-15 14:37:02+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,264.591552,1.0,1.0,264.591552
2,42.09696,14.63285,83.0,2025-04-15 14:37:10+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,81.440227,0.0,1.0,346.031779
3,42.09756,14.63298,84.0,2025-04-15 14:37:15+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,67.573655,1.0,2.0,413.605433
4,42.09814,14.63308,84.0,2025-04-15 14:37:21+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,65.018675,0.0,2.0,478.624108
...,...,...,...,...,...,...,...,...,...,...
5214,42.29458,14.32526,211.0,2025-04-15 17:29:18+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,15.274529,2.0,129.0,150945.776388
5215,42.29416,14.32503,213.0,2025-04-15 17:29:23+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,50.387910,2.0,131.0,150996.164298
5216,42.29380,14.32484,215.0,2025-04-15 17:29:26+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,42.972550,2.0,133.0,151039.136847
5217,42.29370,14.32478,216.0,2025-04-15 17:29:26+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,12.165450,1.0,134.0,151051.302297


In [160]:
profile_df["distance_km"] = profile_df["cum_distance"] / 1000
profile_df

,latitude,longitude,elevation,time,name,link,distance,elevation_diff,cum_elevation,cum_distance,distance_km
0,42.09387,14.63236,82.0,2025-04-15 14:36:39+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,0.000000,0.0,0.0,0.000000,0.000000
1,42.09623,14.63277,83.0,2025-04-15 14:37:02+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,264.591552,1.0,1.0,264.591552,0.264592
2,42.09696,14.63285,83.0,2025-04-15 14:37:10+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,81.440227,0.0,1.0,346.031779,0.346032
3,42.09756,14.63298,84.0,2025-04-15 14:37:15+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,67.573655,1.0,2.0,413.605433,0.413605
4,42.09814,14.63308,84.0,2025-04-15 14:37:21+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,65.018675,0.0,2.0,478.624108,0.478624
...,...,...,...,...,...,...,...,...,...,...,...
5214,42.29458,14.32526,211.0,2025-04-15 17:29:18+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,15.274529,2.0,129.0,150945.776388,150.945776
5215,42.29416,14.32503,213.0,2025-04-15 17:29:23+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,50.387910,2.0,131.0,150996.164298,150.996164
5216,42.29380,14.32484,215.0,2025-04-15 17:29:26+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,42.972550,2.0,133.0,151039.136847,151.039137
5217,42.29370,14.32478,216.0,2025-04-15 17:29:26+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,12.165450,1.0,134.0,151051.302297,151.051302


In [161]:
# Step 1: Make sure both km columns are floats
timeline_df["km"] = timeline_df["km"].astype(float)
timeline_df["km_adjusted"] = timeline_df["km_adjusted"].astype(float)
profile_df["distance_km"] = profile_df["distance_km"].astype(float)

# Step 2: Sort both dataframes by km
timeline_df = timeline_df.sort_values("km_adjusted")
profile_df = profile_df.sort_values("distance_km")

# Step 3: Perform nearest merge
merged_df = pd.merge_asof(
    timeline_df,
    profile_df,
    left_on="km_adjusted",
    right_on="distance_km",
    direction="nearest"
)

# Optional: drop redundant column
merged_df

,timestamp,group_index,average_speed,group_type,km,time_gap,num_riders,riders,time_gap_min,km_adjusted,...,longitude,elevation,time,name,link,distance,elevation_diff,cum_elevation,cum_distance,distance_km
0,20250415T112244Z,2,43.4,Group 3,60.0,+ 04' 20'',1,Large group,4.333333,56.865556,...,14.23446,250.0,2025-04-15 15:49:49+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,12.804596,1.0,168.0,56863.766380,56.863766
1,20250415T112244Z,1,43.4,Group 2,60.0,+ 03' 00'',1,Sampo MALINEN (MIG),3.000000,57.830000,...,14.23171,291.0,2025-04-15 15:50:43+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,17.867372,1.0,209.0,57831.418111,57.831418
2,20250415T112744Z,2,42.3,Group 3,62.0,+ 03' 53'',1,Large group,3.883333,59.262250,...,14.23225,369.0,2025-04-15 15:51:47+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,20.354508,1.0,287.0,59243.668368,59.243668
3,20250415T113016Z,2,42.3,Group 3,62.0,+ 03' 53'',1,Large group,3.883333,59.262250,...,14.23225,369.0,2025-04-15 15:51:47+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,20.354508,1.0,287.0,59243.668368,59.243668
4,20250415T113016Z,1,42.3,Group 2,62.0,+ 03' 12'',1,Sampo MALINEN (MIG),3.200000,59.744000,...,14.22739,404.0,2025-04-15 15:52:16+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,16.517766,1.0,322.0,59747.978946,59.747979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,20250415T132352Z,0,42.4,Group 1,137.0,False,8,"Louis Du Bouisson MEINTJES (IWA), Luca CRETTI ...",0.000000,137.000000,...,14.30065,151.0,2025-04-15 17:15:01+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,19.272613,-1.0,69.0,137009.464853,137.009465
63,20250415T132553Z,0,42.4,Group 1,147.0,False,1,Large group,0.000000,147.000000,...,14.30561,236.0,2025-04-15 17:25:43+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,130.357770,1.0,154.0,147063.055441,147.063055
64,20250415T132753Z,0,42.4,Group 1,147.0,False,1,Large group,0.000000,147.000000,...,14.30561,236.0,2025-04-15 17:25:43+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,130.357770,1.0,154.0,147063.055441,147.063055
65,20250415T132953Z,0,42.7,Group 1,150.0,False,1,Large group,0.000000,150.000000,...,14.32502,157.0,2025-04-15 17:28:35+02:00,giro-abruzzo-2025-stage-1,https://www.la-flamme-rouge.eu/maps/viewtrack/...,19.536645,1.0,75.0,149998.592248,149.998592


In [162]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import datetime

In [163]:
merged_df["km_adjusted"] = pd.to_numeric(merged_df["km_adjusted"], errors="coerce")
merged_df["elevation"] = pd.to_numeric(merged_df["elevation"], errors="coerce")

In [164]:
profile_df["distance_km"] = pd.to_numeric(profile_df["distance_km"], errors="coerce")
profile_df["elevation"] = pd.to_numeric(profile_df["elevation"], errors="coerce")

In [ ]:
#merged_df = merged_df.rename(columns={"tipo": "group_type"})

In [165]:


# Assume df_profile (elevation profile) and df_timeline (group positions over time) already exist

app = dash.Dash(__name__)

timestamps = sorted(merged_df["timestamp"].unique())

app.layout = html.Div([
    html.H2("Race Timeline on Elevation Profile"),
    dcc.Slider(
        id="time-slider",
        min=0,
        max=len(timestamps)-1,
        value=0,
        marks={i: datetime.datetime.fromisoformat(ts).strftime("%H:%M") for i, ts in enumerate(timestamps)},  # show HH:MM
        step=None
    ),
    dcc.Graph(id="elevation-graph")
])

@app.callback(
    Output("elevation-graph", "figure"),
    Input("time-slider", "value")
)
def update_graph(slider_val):
    ts = timestamps[slider_val]
    current = merged_df[merged_df["timestamp"] == ts]

    fig = go.Figure()

    # Elevation profile line
    fig.add_trace(go.Scatter(
        x=profile_df["distance_km"],
        y=profile_df["elevation"],
        mode="lines",
        name="Elevation Profile",
        line=dict(color="darkblue", width=3)
    ))

    # Group icons
    for group_type in current["group_type"].unique():
        group_data = current[current["group_type"] == group_type]
        for _, row in group_data.iterrows():
            fig.add_trace(go.Scatter(
                x=[row["km_adjusted"]],
                y=[row["elevation"]],
                mode="markers+text",
                marker=dict(size=12),
                text=[group_type],
                name=group_type,
                textposition="top center",
                hovertemplate=(
                    "<b>Group:</b> %{text}<br>" +
                    "<b>Gap:</b> " + str(row["time_gap"]) + "<br>" +
                    "<b>Num_riders:</b> " + str(row["num_riders"]) + "<br>" +
                    "<b>Riders:</b><br>" + "<br>".join(row["riders"]) +
                    "<extra></extra>"
                )
            ))

    fig.update_layout(
        xaxis=dict(title="Distance (km)", range=[0, profile_df["distance_km"].max()]),
        yaxis=dict(title="Elevation (m)", range=[0, profile_df["elevation"].max() + 100]),
        title=f"Race State at {datetime.datetime.fromisoformat(ts).strftime("%H:%M")}",
        height=600,
        plot_bgcolor="#ffffff",
        paper_bgcolor="#f7f7f7",
        font=dict(family="Helvetica", size=14),
        margin=dict(t=50, b=50, l=50, r=50),
        legend=dict(
            title=None,
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
        
    )
    fig.update_xaxes(
    showgrid=False,
    gridwidth=1,
    gridcolor="#eeeeee",
    zeroline=False
    )
    fig.update_yaxes(
        showgrid=False,
        gridwidth=1,
        gridcolor="#eeeeee",
        zeroline=False
    )


    return fig

if __name__ == "__main__":
    app.run(debug=True)
